In [1]:
# Manikandan V
# Checked working on Windows 10
# Run the notebook to check the working.
# Code takes care of all the errors mentioned and the other cases

In [2]:
# Links for reference:
# https://en.wikipedia.org/wiki/Key%E2%80%93value_database
# https://www.w3schools.com/js/js_json_objects.asp
# https://en.wikipedia.org/wiki/Thread_safety

In [3]:
import time
import sys
import threading 
from threading import *

**SERVER**

In [4]:
datastore = {}

In [6]:
def create(key,value,timeout=0):                                                              # timeout provided in seconds 
    if key in datastore:
        print("Error :( Key already exists in the datastore.")                                # error msg
    else:
        if(key.isalpha()):                                                                    # string key      #1073741824 bytes ==1 Gb   
            if sys.getsizeof(datastore)<(1073741824) and sys.getsizeof(value)<=(16*1024): # Check file size<=1GB and Json obj size<=16kb 
                if timeout==0:
                    l=[value,-1]
                else:
                    l=[value,time.time()+timeout]                                             # adding timeout incase its not zero
                if len(key)<=32:                                                              # key is max of 32 chars
                    datastore[key]=l 
            else:
                print("Error :( Memory exceeded")                                             # error msg
        else:
            print("Error :( Key should have alphabet only")                                   # error msg

In [7]:
def delete(key):
    if key not in datastore:
        print("Error :( Key is not in Datastore")                                             # error msg
    else:
        list=datastore[key]
        if list[1]!=-1:    # time to live parameter isnt -1(means its provided by user)
            current_time=time.time()
            if current_time<list[1]:                                                          # Expiry & current time compared
                del datastore[key]
                print("Success :) key is now been deleted")
            else:
                print("Error :( time to live off has been expired")     # error as time to live has expired so cant delete it
        else:                                                           # time to live is -1 then just delete the key
            del datastore[key]
            print("Success! key is now deleted")

In [8]:
def read(key):
    if key not in datastore:
        print("Error :( Key is not in Datastore")                                             # error msg
    else:
        list=datastore[key]
        if list[1]!=-1:                # time to live parameter isnt -1(means it's provided by user)
            current_time=time.time()
            if current_time<list[1]:                                                          # Expiry & current time compared
                response=str(key)+" : "+str(list[0])                                          # Key - JSon pair returned from DB
                return response
            else:
                print("Error :( time to live off has been expired")                           # error msg
        else:
            response =str(key)+" : "+str(list[0])
            return response

**CLIENT**

***Testcase-1***

In [9]:
json1={  "Name": "Manikantan",
  "Rollno": "EC17B1033",
  "Marks": 95}

create("Student",json1)                                      # to create a key with key & json obj given and no time-to-live property

json2=[28,7]
create("Number",json2,200)# to create a key with key & json obj given and with time-to-live property value given(number of seconds)

print(read("Student"))                                                # PRINTS key in Json object format 'key_name:value'

print(read("Number")) # PRINTS key in Json object format 'key_name:value' if the (time to live) is not expired else it throws an ERROR

json3={"Nameid":"lala123"}
create("Student",json3)
# this returns an error since the key_name already present in datastore
 
delete("Student")
# this deletes the given key & json obj from datastore

# #Using Multi threading
json4=["Merry","Christmas"]
thread1=Thread(target=(create),args=("moker",json4))                  # as per the operation
thread1.start()
thread2=Thread(target=(delete),args=("moker",))                       # as per the operation
thread2.start()

print("Final datastore",datastore)

Student : {'Name': 'Manikantan', 'Rollno': 'EC17B1033', 'Marks': 95}
Number : [28, 7]
Error :( Key already exists in the datastore.
Success! key is now deleted
Success! key is now deleted
Final datastore {'Number': [[28, 7], 1609323402.4082234]}


***Testcase-2***

In [10]:
delete("lalalala")

json1={  "Name": "Manikantan",
  "Rollno": "EC17B1033",
  "Marks": 95}

create("Student1",json1)
#Error! as alphanumeric key with key & json obj given and no time-to-live property

json2=[28,7]
create("Numbers",json2,30) 
#to create a key with key & json obj given and with time-to-live property value given(number of seconds) as just 10 secs

print(read("Numbers"))
#PRINTS key in Json object format 'key_name:value' if the (time to live) is not expired else it throws an ERROR !

json3={"Nameid":"lala123"}
create("Employee",json3)

print("Final datastore",datastore)

Error :( Key is not in Datastore
Error :( Key should have alphabet only
Numbers : [28, 7]
Final datastore {'Number': [[28, 7], 1609323402.4082234], 'Numbers': [[28, 7], 1609323320.8328054], 'Employee': [{'Nameid': 'lala123'}, -1]}


In [11]:
print(read("Numbers"))

Error :( time to live off has been expired
None
